In [1]:
import pandas as pd
import numpy as np

### Model 1.1.SVM:
 - Input Features: generic_name (reduced Dim), drug_char, drug_indication (reduced Dim), admin route, reaction_medDRA at SOC level
 - Model: Support Vector Machines, C=1.0, radial basal function for the kernel
 - Input Data: 2008, Quarter 2
 
Too slow to run the higher feature sets.
 
(Convention: Model_X_Y_Z where X is data input size (rows), Y is feature size (cols), and Z is model type (e.g. RF))

In [2]:
test_col = 'seriousness_lifethreatening'
new_col_name = 'Model 1.1 (Life Threatening)'
file_to_read = 'progress/modeling/df_ML_model_1_1.csv'
#file_for_results = 'progress/modeling/model_1_1_SVM_results.csv'
firstrun = False

In [3]:
# Read in the data from Wrangling
df = pd.read_csv(file_to_read, index_col = 0)
# Drop all outcomes except that of interest
all_cols = ['serious', 'seriousness_congential_anomali', 'seriousness_death', 
              'seriousness_disabling', 'seriousness_hospitalization', 
              'seriousness_lifethreatening', 'seriousness_other']

drop_cols = [col for col in all_cols if col != test_col]
df = df.drop(drop_cols, axis = 1)

df.head()

,ABACAVIR,ABACAVIR SULFATE,ACETAMINOPHEN AND CODEINE,ADENOSINE,ALCOHOL,ALENDRONATE SODIUM,ALISKIREN HEMIFUMARATE,ALUMINUM HYDROXIDE AND MA,AMLODIPINE BESYLATE AND O,ANTI-THYMOCYTE GLOBULIN (,...,PRODUCT ISSUES,PSYCHIATRIC DISORDERS,RENAL AND URINARY DISORDERS,REPRODUCTIVE SYSTEM AND BREAST DISORDERS,RESPIRATORY,SKIN AND SUBCUTANEOUS TISSUE DISORDERS,SOCIAL CIRCUMSTANCES,SURGICAL AND MEDICAL PROCEDURES,VASCULAR DISORDERS,seriousness_lifethreatening
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [4]:
from sklearn.model_selection import train_test_split

/home/spwhite1337/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [5]:
X_train, X_test, y_train, y_test = train_test_split(df.drop([test_col], axis = 1), 
                                                    df[test_col].astype('category'), 
                                                    test_size = 0.33, 
                                                    random_state = 189)

In [6]:
# Load scikit's SVM-C library
from sklearn.svm import SVC

In [7]:
# Create a SVM Classifier.
svc = SVC(C = 1.0, kernel = 'rbf', random_state = 189)

In [8]:
# Train the Classifier
svc.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=189, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
# Apply the Classifier we trained to the train and test data 
preds_train = svc.predict(X_train)
preds_test = svc.predict(X_test)

In [10]:
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

In [11]:
# Confusion matrix for training set
cm_train = confusion_matrix(y_train, preds_train)
cm_train = pd.DataFrame(cm_train)
cm_train.columns = ['Negative', 'Positive']
cm_train.index = cm_train.columns
print("Confusion Matrix (Train):")
cm_train

Confusion Matrix (Train):


,Negative,Positive
Negative,44773,0
Positive,1781,0


In [12]:
# Confusion matrix for test set
cm_test = confusion_matrix(y_test, preds_test)
cm_test = pd.DataFrame(cm_test)
cm_test.columns = ['Negative', 'Positive']
cm_test.index = cm_test.columns
print("Confusion Matrix (Test):")
cm_test

Confusion Matrix (Test):


,Negative,Positive
Negative,21989,0
Positive,942,0


In [13]:
# Model accuracy
acc_train = np.trace(np.asarray(cm_train))/len(X_train)
acc_test = np.trace(np.asarray(cm_test))/len(X_test)
fraction_pos = sum(y_train)/len(y_train)
print('Train Accuracy is {a:0.2f}%'.format(a = acc_train*100))
print('Test Accuracy is {a:0.2f}%'.format(a = acc_test*100))
print('Percent Positive: {a:0.2f}%'.format(a = fraction_pos*100))

Train Accuracy is 96.17%
Test Accuracy is 95.89%
Percent Positive: 3.83%


In [14]:
# Output metrics for train set
prfs_train = precision_recall_fscore_support(y_train, preds_train)
prfs_train = np.array([prfs_train[0], prfs_train[1], prfs_train[2], prfs_train[3]])
prfs_train = pd.DataFrame(prfs_train)
prfs_train.columns = ['Negative', 'Positive']
prfs_train.index = ['Precision','Recall','F-score','Support']
print("Precision, Recall, F-Score (Train):")
prfs_train

Precision, Recall, F-Score (Train):


/home/spwhite1337/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Negative,Positive
Precision,0.961743,0.0
Recall,1.000000,0.0
F-score,0.980499,0.0
Support,44773.000000,1781.0


In [15]:
# Output metrics for train set
prfs_test = precision_recall_fscore_support(y_test, preds_test)
prfs_test = np.array([prfs_test[0], prfs_test[1], prfs_test[2], prfs_test[3]])
prfs_test = pd.DataFrame(prfs_test)
prfs_test.columns = ['Negative', 'Positive']
prfs_test.index = ['Precision','Recall','F-score','Support']
print("Precision, Recall, F-Score (Test):")
prfs_test

Precision, Recall, F-Score (Test):


/home/spwhite1337/anaconda3/envs/insight/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Negative,Positive
Precision,0.958920,0.0
Recall,1.000000,0.0
F-score,0.979029,0.0
Support,21989.000000,942.0


In [16]:
# View a list of the features and their relevance
svc.support_

array([    6,    16,    24, ..., 46539, 46546, 46551], dtype=int32)

In [17]:
# View a list of the features and their importance scores
svc.support_vectors_

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Save Results

In [18]:
if firstrun:
    results_prev = pd.DataFrame()
else:
    results_prev = pd.read_csv(file_for_results, index_col = 0)
results_prev

,Model 1.1 (Serious),Model 1.1 (Death),Model 1.1 (Disabling),Model 1.1 (Hospital)
names,,,,
Train_acc,80.117713,91.253168,97.581303,79.148945
Train_Recall_0,0.834171,1.000000,1.000000,0.975636
Train_Recall_1,0.752923,0.001716,0.000887,0.214019
Train_Precision_0,0.831585,0.912519,0.975813,0.795610
Train_Precision_1,0.756363,1.000000,1.000000,0.736923
Train_Fscore_0,0.832876,0.954258,0.987758,0.876475
Train_Fscore_1,0.754639,0.003426,0.001773,0.331704
Test_acc,79.931098,90.933671,97.601500,78.579216
Test_Recall_0,0.833873,1.000000,1.000000,0.974346


In [19]:
# Set up new results to add in a list
results_new = [
    acc_train*100, 
    prfs_train['Negative'][1], prfs_train['Positive'][1], 
    prfs_train['Negative'][0], prfs_train['Positive'][0], 
    prfs_train['Negative'][2], prfs_train['Positive'][2], 
    acc_test*100, 
    prfs_test['Negative'][1], prfs_test['Positive'][1], 
    prfs_test['Negative'][0], prfs_test['Positive'][0], 
    prfs_test['Negative'][2], prfs_test['Positive'][2], 
    fraction_pos*100
]

In [20]:
# add the new results
results = results_prev
results[new_col_name] = results_new

# Set index
if firstrun:
    results['names'] = pd.DataFrame(['Train_acc', 
                                     'Train_Recall_0', 'Train_Recall_1', 'Train_Precision_0',
                                     'Train_Precision_1', 'Train_Fscore_0', 'Train_Fscore_1', 
                                     'Test_acc', 
                                     'Test_Recall_0', 'Test_Recall_1','Test_Precision_0', 
                                     'Test_Precision_1', 'Test_Fscore_0', 'Test_Fscore_1', 
                                     'Percent_positive'])
    results = results.set_index('names')
# Show
results

,Model 1.1 (Serious),Model 1.1 (Death),Model 1.1 (Disabling),Model 1.1 (Hospital),Model 1.1 (Life Threatening)
names,,,,,
Train_acc,80.117713,91.253168,97.581303,79.148945,96.174335
Train_Recall_0,0.834171,1.000000,1.000000,0.975636,1.000000
Train_Recall_1,0.752923,0.001716,0.000887,0.214019,0.000000
Train_Precision_0,0.831585,0.912519,0.975813,0.795610,0.961743
Train_Precision_1,0.756363,1.000000,1.000000,0.736923,0.000000
Train_Fscore_0,0.832876,0.954258,0.987758,0.876475,0.980499
Train_Fscore_1,0.754639,0.003426,0.001773,0.331704,0.000000
Test_acc,79.931098,90.933671,97.601500,78.579216,95.892024
Test_Recall_0,0.833873,1.000000,1.000000,0.974346,1.000000


In [21]:
# Save it
results.to_csv(file_for_results)